# Trading Strategy Testing Notebook

This notebook allows you to test various components of the trading strategy framework:

1. Loading data from Tiingo
2. Testing entry strategies
3. Testing exit strategies
4. Visualizing results
5. Implementing new strategies


## Setup and Imports

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Add the project root to the path
sys.path.append(os.path.dirname(os.path.abspath('.')))

# Import project modules
from utils.data_loader import DataLoader
from utils.indicators import Indicators
from utils.stats import compute_returns, compute_sharpe, compute_pnl_spark, compute_total_return, compute_drawdown
from strategies.entries.moving_average_crossover import MovingAverageCrossover
from strategies.exits.exit_trailing_stop import ExitTrailingStop
from config import SYMBOLS, STRATEGY_CONFIG, API_CONFIG

# Set up matplotlib for better visualization
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 12

## 1. Loading Data from Tiingo

First, let's set up the data loader and fetch some historical data.

In [ ]:
# Initialize data loader with your Tiingo API key
# You can either set it in config.py, as an environment variable, or directly here
api_key = None  # Replace with your API key if not set in config.py or .env file
data_loader = DataLoader(api_key=api_key)

# Set date range for historical data
end_date = datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')  # 1 year of data

# Define symbols to analyze
symbols = ['AAPL', 'MSFT', 'SPY']  # Example symbols

# Fetch historical data
data = data_loader.get_multiple_symbols(symbols, start_date=start_date, end_date=end_date)

# Check if data was loaded successfully
for symbol, df in data.items():
    print(f'{symbol}: {len(df)} days of data from {df.index.min().date()} to {df.index.max().date()}')

In [ ]:
# Let's look at the first few rows of data for one symbol
if 'AAPL' in data:
    data['AAPL'].head()

## 2. Testing Entry Strategies

Let's test the Moving Average Crossover entry strategy.

In [ ]:
# Initialize the Moving Average Crossover strategy
ma_crossover = MovingAverageCrossover(fast_period=20, slow_period=50)

# Generate signals for a symbol
symbol = 'AAPL'  # Change to any symbol in your data
if symbol in data:
    df_with_signals = ma_crossover.generate_signal(data[symbol])
    
    # Display the latest signal
    latest_signal = ma_crossover.get_latest_signal(data[symbol])
    print(f'Latest signal for {symbol}:
{latest_signal}')
    
    # Plot the price and moving averages
    plt.figure(figsize=(14, 10))
    
    # Plot price
    plt.subplot(2, 1, 1)
    plt.plot(df_with_signals.index, df_with_signals['close'], label='Close Price')
    plt.plot(df_with_signals.index, df_with_signals[f'SMA_{ma_crossover.fast_period}'], label=f'SMA {ma_crossover.fast_period}')
    plt.plot(df_with_signals.index, df_with_signals[f'SMA_{ma_crossover.slow_period}'], label=f'SMA {ma_crossover.slow_period}')
    
    # Plot buy signals
    buy_signals = df_with_signals[df_with_signals['entry_long'] == 1]
    plt.scatter(buy_signals.index, buy_signals['close'], color='green', marker='^', s=100, label='Buy Signal')
    
    # Plot sell signals
    sell_signals = df_with_signals[df_with_signals['entry_short'] == 1]
    plt.scatter(sell_signals.index, sell_signals['close'], color='red', marker='v', s=100, label='Sell Signal')
    
    plt.title(f'{symbol} - Moving Average Crossover Strategy')
    plt.ylabel('Price')
    plt.legend()
    
    # Plot position
    plt.subplot(2, 1, 2)
    plt.plot(df_with_signals.index, df_with_signals['signal'], label='Position (1=Long, -1=Short, 0=Flat)')
    plt.title(f'{symbol} - Position')
    plt.ylabel('Position')
    plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    plt.legend()
    
    plt.tight_layout()
    plt.show()

## 3. Testing Exit Strategies

Now let's test the Trailing Stop exit strategy.

In [ ]:
# Initialize the Trailing Stop exit strategy
trailing_stop = ExitTrailingStop(atr_period=14, atr_multiplier=2.0)

# Generate signals for a symbol
symbol = 'AAPL'  # Change to any symbol in your data
if symbol in data:
    df_with_exits = trailing_stop.generate_signal(data[symbol])
    
    # Display the latest signal
    latest_exit = trailing_stop.get_latest_signal(data[symbol], position_type='long')
    print(f'Latest exit signal for {symbol} (long position):
{latest_exit}')
    
    # Plot the price and trailing stops
    plt.figure(figsize=(14, 10))
    
    # Plot price and trailing stops
    plt.subplot(2, 1, 1)
    plt.plot(df_with_exits.index, df_with_exits['close'], label='Close Price')
    plt.plot(df_with_exits.index, df_with_exits['trailing_stop_long'], label='Trailing Stop (Long)', color='red', linestyle='--')
    
    # Plot exit signals
    exit_signals = df_with_exits[df_with_exits['exit_long'] == 1]
    plt.scatter(exit_signals.index, exit_signals['close'], color='red', marker='x', s=100, label='Exit Signal')
    
    plt.title(f'{symbol} - Trailing Stop Exit Strategy (Long Position)')
    plt.ylabel('Price')
    plt.legend()
    
    # Plot ATR
    plt.subplot(2, 1, 2)
    plt.plot(df_with_exits.index, df_with_exits['ATR'], label='ATR')
    plt.plot(df_with_exits.index, df_with_exits['stop_distance'], label='Stop Distance (ATR * multiplier)')
    plt.title(f'{symbol} - ATR and Stop Distance')
    plt.ylabel('Value')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

## 4. Combining Entry and Exit Strategies

Let's combine the entry and exit strategies to create a complete trading system.

In [ ]:
def backtest_strategy(symbol_data, entry_strategy, exit_strategy, initial_capital=10000):
    # Make a copy of the data
    df = symbol_data.copy()
    
    # Generate entry signals
    df = entry_strategy.generate_signal(df)
    
    # Generate exit signals
    df = exit_strategy.generate_signal(df)
    
    # Initialize columns for backtesting
    df['position'] = 0  # 0 = no position, 1 = long, -1 = short
    df['entry_price'] = 0.0
    df['exit_price'] = 0.0
    df['trade_return'] = 0.0
    df['equity'] = initial_capital
    
    # Simulate trading
    position = 0
    entry_price = 0.0
    
    for i in range(1, len(df)):
        # Update position based on previous day's signals
        if position == 0:  # No position
            if df.iloc[i-1]['entry_long'] == 1:  # Entry signal for long
                position = 1
                entry_price = df.iloc[i]['open']  # Enter at next day's open
            elif df.iloc[i-1]['entry_short'] == 1:  # Entry signal for short
                position = -1
                entry_price = df.iloc[i]['open']  # Enter at next day's open
        elif position == 1:  # Long position
            if df.iloc[i-1]['exit_long'] == 1:  # Exit signal for long
                # Calculate return
                exit_price = df.iloc[i]['open']  # Exit at next day's open
                df.loc[df.index[i], 'trade_return'] = (exit_price / entry_price) - 1
                df.loc[df.index[i], 'exit_price'] = exit_price
                position = 0
        elif position == -1:  # Short position
            if df.iloc[i-1]['exit_short'] == 1:  # Exit signal for short
                # Calculate return
                exit_price = df.iloc[i]['open']  # Exit at next day's open
                df.loc[df.index[i], 'trade_return'] = 1 - (exit_price / entry_price)  # Short return
                df.loc[df.index[i], 'exit_price'] = exit_price
                position = 0
        
        # Update position and entry price
        df.loc[df.index[i], 'position'] = position
        if position != 0 and df.loc[df.index[i-1], 'position'] != position:
            df.loc[df.index[i], 'entry_price'] = entry_price
        
        # Update equity
        if df.loc[df.index[i], 'trade_return'] != 0:
            df.loc[df.index[i], 'equity'] = df.loc[df.index[i-1], 'equity'] * (1 + df.loc[df.index[i], 'trade_return'])
        else:
            df.loc[df.index[i], 'equity'] = df.loc[df.index[i-1], 'equity']
    
    # Calculate cumulative returns
    df['cum_return'] = df['equity'] / initial_capital - 1
    
    # Calculate drawdown
    df['equity_peak'] = df['equity'].cummax()
    df['drawdown'] = (df['equity'] / df['equity_peak']) - 1
    
    return df

# Run backtest for a symbol
symbol = 'AAPL'  # Change to any symbol in your data
if symbol in data:
    # Initialize strategies
    entry_strategy = MovingAverageCrossover(fast_period=20, slow_period=50)
    exit_strategy = ExitTrailingStop(atr_period=14, atr_multiplier=2.0)
    
    # Run backtest
    backtest_results = backtest_strategy(data[symbol], entry_strategy, exit_strategy)
    
    # Calculate performance metrics
    total_return = backtest_results['cum_return'].iloc[-1]
    max_drawdown = backtest_results['drawdown'].min()
    sharpe = compute_sharpe(backtest_results, periods=[len(backtest_results)])[f'{len(backtest_results)}d']
    
    print(f'Backtest Results for {symbol}:
')
    print(f'Total Return: {total_return:.2%}')
    print(f'Max Drawdown: {max_drawdown:.2%}')
    print(f'Sharpe Ratio: {sharpe:.2f}')
    
    # Plot equity curve and drawdown
    plt.figure(figsize=(14, 10))
    
    # Plot equity curve
    plt.subplot(2, 1, 1)
    plt.plot(backtest_results.index, backtest_results['equity'], label='Equity Curve')
    plt.title(f'{symbol} - Equity Curve')
    plt.ylabel('Equity ($)')
    plt.legend()
    
    # Plot drawdown
    plt.subplot(2, 1, 2)
    plt.fill_between(backtest_results.index, backtest_results['drawdown'] * 100, 0, color='red', alpha=0.3)
    plt.plot(backtest_results.index, backtest_results['drawdown'] * 100, color='red', label='Drawdown %')
    plt.title(f'{symbol} - Drawdown')
    plt.ylabel('Drawdown (%)')
    plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    plt.legend()
    
    plt.tight_layout()
    plt.show()

## 5. Implementing New Strategies

Let's implement some of the other strategies mentioned in the requirements.

### 5.1 Momentum Breakout Entry Strategy

In [ ]:
# Define the Momentum Breakout strategy class
class MomentumBreakout:
    def __init__(self, lookback_period=20, volatility_factor=1.5):
        self.name = "Momentum Breakout"
        self.lookback_period = lookback_period
        self.volatility_factor = volatility_factor
        self.tags = ["momentum", "breakout", "volatility"]
    
    def generate_signal(self, df):
        """Generate entry signals based on price breakouts"""
        # Make a copy to avoid modifying the original
        df = df.copy()
        
        # Add ATR for volatility measurement
        df = Indicators.add_atr(df, period=self.lookback_period)
        
        # Calculate upper and lower breakout levels
        df['high_max'] = df['high'].rolling(window=self.lookback_period).max()
        df['low_min'] = df['low'].rolling(window=self.lookback_period).min()
        
        # Calculate breakout thresholds with ATR volatility adjustment
        df['upper_threshold'] = df['high_max'] + (df['ATR'] * self.volatility_factor)
        df['lower_threshold'] = df['low_min'] - (df['ATR'] * self.volatility_factor)
        
        # Initialize signal columns
        df['signal'] = 0
        df['entry_long'] = 0
        df['entry_short'] = 0
        
        # Generate breakout signals
        for i in range(self.lookback_period + 1, len(df)):
            # Long signal: price breaks above upper threshold
            if df.iloc[i]['close'] > df.iloc[i-1]['upper_threshold']:
                df.loc[df.index[i], 'entry_long'] = 1
                df.loc[df.index[i], 'signal'] = 1
            
            # Short signal: price breaks below lower threshold
            elif df.iloc[i]['close'] < df.iloc[i-1]['lower_threshold']:
                df.loc[df.index[i], 'entry_short'] = 1
                df.loc[df.index[i], 'signal'] = -1
        
        return df
    
    def get_latest_signal(self, df):
        """Get the latest signal from the dataframe"""
        df = self.generate_signal(df)
        latest = df.iloc[-1]
        
        return {
            "entry_long": bool(latest['entry_long']),
            "entry_short": bool(latest['entry_short']),
            "current_position": int(latest['signal']),
            "indicators": {
                "ATR": latest['ATR'],
                "upper_threshold": latest['upper_threshold'],
                "lower_threshold": latest['lower_threshold']
            }
        }

# Test the Momentum Breakout strategy
momentum_breakout = MomentumBreakout(lookback_period=20, volatility_factor=1.5)

# Generate signals for a symbol
symbol = 'AAPL'  # Change to any symbol in your data
if symbol in data:
    df_with_signals = momentum_breakout.generate_signal(data[symbol])
    
    # Display the latest signal
    latest_signal = momentum_breakout.get_latest_signal(data[symbol])
    print(f'Latest signal for {symbol} (Momentum Breakout):
{latest_signal}')
    
    # Plot the price and breakout levels
    plt.figure(figsize=(14, 10))
    
    # Plot price and breakout levels
    plt.plot(df_with_signals.index, df_with_signals['close'], label='Close Price')
    plt.plot(df_with_signals.index, df_with_signals['upper_threshold'], label='Upper Threshold', color='green', linestyle='--')
    plt.plot(df_with_signals.index, df_with_signals['lower_threshold'], label='Lower Threshold', color='red', linestyle='--')
    
    # Plot buy signals
    buy_signals = df_with_signals[df_with_signals['entry_long'] == 1]
    plt.scatter(buy_signals.index, buy_signals['close'], color='green', marker='^', s=100, label='Buy Signal')
    
    # Plot sell signals
    sell_signals = df_with_signals[df_with_signals['entry_short'] == 1]
    plt.scatter(sell_signals.index, sell_signals['close'], color='red', marker='v', s=100, label='Sell Signal')
    
    plt.title(f'{symbol} - Momentum Breakout Strategy')
    plt.ylabel('Price')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

### 5.2 RSI Overbought/Oversold Exit Strategy

In [ ]:
# Define the RSI Overbought/Oversold Exit strategy class
class ExitRsiOverbought:
    def __init__(self, rsi_period=14, overbought=70, oversold=30):
        self.name = "RSI Overbought/Oversold Exit"
        self.rsi_period = rsi_period
        self.overbought = overbought
        self.oversold = oversold
        self.tags = ["mean_reversion", "oscillator", "overbought"]
    
    def generate_signal(self, df, entry_price=None):
        """Generate exit signals based on RSI overbought/oversold levels"""
        # Make a copy to avoid modifying the original
        df = df.copy()
        
        # Add RSI indicator
        df = Indicators.add_rsi(df, period=self.rsi_period)
        
        # Generate exit signals
        df['exit_long'] = (df['RSI'] > self.overbought).astype(int)
        df['exit_short'] = (df['RSI'] < self.oversold).astype(int)
        
        return df
    
    def get_latest_signal(self, df, position_type="long", entry_price=None):
        """Get the latest signal from the dataframe"""
        df = self.generate_signal(df, entry_price)
        latest = df.iloc[-1]
        
        if position_type.lower() == "long":
            exit_signal = bool(latest['exit_long'])
        else:  # short
            exit_signal = bool(latest['exit_short'])
        
        return {
            "exit_signal": exit_signal,
            "indicators": {
                "RSI": latest['RSI'],
                "overbought_level": self.overbought,
                "oversold_level": self.oversold
            }
        }

# Test the RSI Overbought/Oversold Exit strategy
rsi_exit = ExitRsiOverbought(rsi_period=14, overbought=70, oversold=30)

# Generate signals for a symbol
symbol = 'AAPL'  # Change to any symbol in your data
if symbol in data:
    df_with_exits = rsi_exit.generate_signal(data[symbol])
    
    # Display the latest signal
    latest_exit = rsi_exit.get_latest_signal(data[symbol], position_type='long')
    print(f'Latest exit signal for {symbol} (RSI Overbought/Oversold - Long Position):
{latest_exit}')
    
    # Plot the price and RSI
    plt.figure(figsize=(14, 10))
    
    # Plot price
    plt.subplot(2, 1, 1)
    plt.plot(df_with_exits.index, df_with_exits['close'], label='Close Price')
    
    # Plot exit signals for long positions
    exit_signals = df_with_exits[df_with_exits['exit_long'] == 1]
    plt.scatter(exit_signals.index, exit_signals['close'], color='red', marker='x', s=100, label='Exit Long Signal')
    
    plt.title(f'{symbol} - RSI Overbought/Oversold Exit Strategy')
    plt.ylabel('Price')
    plt.legend()
    
    # Plot RSI
    plt.subplot(2, 1, 2)
    plt.plot(df_with_exits.index, df_with_exits['RSI'], label='RSI')
    plt.axhline(y=rsi_exit.overbought, color='red', linestyle='--', label=f'Overbought ({rsi_exit.overbought})')
    plt.axhline(y=rsi_exit.oversold, color='green', linestyle='--', label=f'Oversold ({rsi_exit.oversold})')
    plt.title(f'{symbol} - RSI')
    plt.ylabel('RSI Value')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

## 6. Comparing Multiple Strategies

Let's compare the performance of different strategy combinations.

In [ ]:
# Define strategy combinations to test
strategy_combinations = [
    {
        'name': 'MA Crossover + Trailing Stop',
        'entry': MovingAverageCrossover(fast_period=20, slow_period=50),
        'exit': ExitTrailingStop(atr_period=14, atr_multiplier=2.0)
    },
    {
        'name': 'MA Crossover + RSI Exit',
        'entry': MovingAverageCrossover(fast_period=20, slow_period=50),
        'exit': ExitRsiOverbought(rsi_period=14, overbought=70, oversold=30)
    },
    {
        'name': 'Momentum Breakout + Trailing Stop',
        'entry': MomentumBreakout(lookback_period=20, volatility_factor=1.5),
        'exit': ExitTrailingStop(atr_period=14, atr_multiplier=2.0)
    },
    {
        'name': 'Momentum Breakout + RSI Exit',
        'entry': MomentumBreakout(lookback_period=20, volatility_factor=1.5),
        'exit': ExitRsiOverbought(rsi_period=14, overbought=70, oversold=30)
    }
]

# Run backtest for each strategy combination
symbol = 'AAPL'  # Change to any symbol in your data
if symbol in data:
    # Store results for comparison
    results = {}
    
    for strategy in strategy_combinations:
        print(f"Running backtest for {strategy['name']}...")
        backtest_results = backtest_strategy(
            data[symbol], 
            strategy['entry'], 
            strategy['exit']
        )
        
        # Calculate performance metrics
        total_return = backtest_results['cum_return'].iloc[-1]
        max_drawdown = backtest_results['drawdown'].min()
        sharpe = compute_sharpe(backtest_results, periods=[len(backtest_results)])[f'{len(backtest_results)}d']
        
        # Store results
        results[strategy['name']] = {
            'equity_curve': backtest_results['equity'],
            'total_return': total_return,
            'max_drawdown': max_drawdown,
            'sharpe_ratio': sharpe
        }
    
    # Display comparison table
    comparison = pd.DataFrame({
        'Strategy': list(results.keys()),
        'Total Return': [results[s]['total_return'] for s in results],
        'Max Drawdown': [results[s]['max_drawdown'] for s in results],
        'Sharpe Ratio': [results[s]['sharpe_ratio'] for s in results]
    })
    
    print("\nStrategy Comparison:")
    print(comparison.to_string(index=False, formatters={
        'Total Return': '{:.2%}'.format,
        'Max Drawdown': '{:.2%}'.format,
        'Sharpe Ratio': '{:.2f}'.format
    }))
    
    # Plot equity curves for comparison
    plt.figure(figsize=(14, 8))
    
    for strategy_name, result in results.items():
        plt.plot(result['equity_curve'].index, result['equity_curve'], label=strategy_name)
    
    plt.title(f'{symbol} - Strategy Comparison')
    plt.ylabel('Equity ($)')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

## 7. Exporting Results

Let's export the results to a JSON file for further analysis.

In [ ]:
import json
from utils.json_export import export_portfolio

# Prepare portfolio data for export
portfolio_data = {}

# Run the best strategy for multiple symbols
symbols_to_analyze = ['AAPL', 'MSFT', 'SPY']  # Add more symbols as needed
best_strategy = strategy_combinations[0]  # Use the first strategy as an example, or pick the best one

for symbol in symbols_to_analyze:
    if symbol in data:
        # Run backtest
        backtest_results = backtest_strategy(
            data[symbol], 
            best_strategy['entry'], 
            best_strategy['exit']
        )
        
        # Calculate performance metrics
        total_return = backtest_results['cum_return'].iloc[-1]
        max_drawdown = backtest_results['drawdown'].min()
        returns = compute_returns(data[symbol])
        sharpe_ratios = compute_sharpe(data[symbol])
        position_size = 10000  # Example position size
        pnl_spark = compute_pnl_spark(data[symbol], position_size)
        
        # Get latest signals
        latest_entry = best_strategy['entry'].get_latest_signal(data[symbol])
        latest_exit = best_strategy['exit'].get_latest_signal(data[symbol])
        
        # Store in portfolio data
        portfolio_data[symbol] = {
            "symbol": symbol,
            "tags": next((s["tags"] for s in SYMBOLS if isinstance(s, dict) and s["symbol"] == symbol), []),
            "allocation": STRATEGY_CONFIG["default_allocation"],
            "exit_signal": latest_exit["exit_signal"],
            "entry_signal": latest_entry["entry_long"],
            "latest_indicators": {
                "close": data[symbol]["close"].iloc[-1],
                **latest_entry["indicators"],
                **latest_exit["indicators"]
            },
            "position_size_dollars": position_size,
            "pnl_spark_chart": pnl_spark,
            "returns": returns,
            "sharpe_ratios": sharpe_ratios,
            "total_return": total_return,
            "max_drawdown": max_drawdown,
            "total_pnl_dollars": position_size * total_return
        }

# Export portfolio data to JSON file
output_path = export_portfolio(portfolio_data, output_path="../outputs/portfolio.json")
print(f"Portfolio data exported to {output_path}")